In [1]:
# # 邮件配置
# from utils.MailSenderHTML import MailSenderHTML
# mail_filename = 'config/mail.json'
# msh = MailSenderHTML(mail_filename)

In [1]:
# 每日回顾
import sys
# sys.path.append("../datasource/DailyReview")

sys.path.append("../../")

from EMData import EMData
from WSData import WSData
from CH_VIX import CH_VIX

from Nations import Nations
from Liquidity import Liquidity
from Leverage import Leverage
from Economic import Economic
from Direction import Direction
from Sentiment import Sentiment

from MacroViewData import MacroViewData


from datetime import datetime,timedelta
import warnings
warnings.filterwarnings("ignore")

In [2]:
def update_img_data_dict(img_data_dict,fig,name):
    filename = result_path + '{}.png'.format(name)
    fig.savefig(filename,bbox_inches = 'tight', dpi=70)
    file = open(filename, "rb")
    img_data = file.read()
    file.close()
    img_data_dict[name] = img_data
    
    return img_data_dict

def repalce_message(html_content,content,name,isEmail = True):
    if isEmail:
        html_content = html_content.replace('替换{}图片'.format(name),'<img src="cid:{0}" alt="{0}">'.format(name))
    else:
#         html_content = html_content.replace('替换{}图片'.format(name),'<img src="/home/jquser/0.定时推送/datasource/DailyReview/data/{0}.png" alt="{0}">'.format(name))
        html_content = html_content.replace('替换{}图片'.format(name),'<img src="{0}.png" alt="{0}">'.format(name))
    html_content = html_content.replace('替换{}表格'.format(name),content)
    
    return html_content



In [3]:
# data_path = './datasource/DailyReview/data/'
path = '../'
data_path = path + 'process/'
result_path = path + 'result/'
em = EMData(data_path)
mvd = MacroViewData(data_path)
ws = WSData(data_path)

try:
    # 中国PPI CPI PMI 北向资金
    em.save_data(typeStr = 'ppi')
    em.save_data(typeStr = 'cpi')
    em.save_data(typeStr = 'pmi')
    em.save_data(typeStr = 'north_money')
    
    # 美国PMI
    em.save_data(typeStr = 'us_manufacturing_pmi')
    em.save_data(typeStr = 'us_non_manufacturing_pmi')
    
    # 中美利差
    mvd.save_data('us_fed_rate') 
    mvd.save_data('cn_dr007')
    
    # 美国CPI
    mvd.save_data('us_cpi')
    
    # 美国国债利差：10年-2年
    mvd.save_data('us_bond_10Y-2Y')
    
    # 信贷
    em.save_data(typeStr = 'loan')
    
    

    ws.save_data(typeStr = 'USDCNY', count = 30)
    ws.save_data(typeStr = 'USDJPY', count = 30)
    ws.save_data(typeStr = 'US10YR', count = 30)
    ws.save_data(typeStr = 'USCL', count = 30)
    ws.save_data(typeStr = 'CN10YR', count = 30)

except Exception as e:
    # 访问异常的错误编号和详细信息
    print(e.args)
    print("获取数据出错。")

In [4]:
start_date = '2015-01-01'
end_date = datetime.now().strftime("%Y-%m-%d")

try:
    #上证50波幅指数
    VIX = CH_VIX('510050.XSHG',start_date,end_date,data_path)
    VIX_ser = VIX.GetCBOE_VIX()

    # 沪深300波幅指数
    VIX = CH_VIX('510300.XSHG',start_date,end_date,data_path)
    VIX_ser = VIX.GetCBOE_VIX()

    # 中证1000波幅指数
    VIX = CH_VIX('000852.XSHG',start_date,end_date,data_path)
    VIX_ser = VIX.GetCBOE_VIX()
except Exception as e:
    # 访问异常的错误编号和详细信息
    print(e.args)


In [5]:
# def main():
# 读取模板文件
filename = path + 'content.html'
with open(filename,'rb') as f:
    html_content = f.read().decode('GBK')
    msg_content = html_content

img_data_dict = {}  


In [6]:
# 1 国别资金流
na = Nations(data_path)

## 美元人民币
fig,content = na.show_exchange_data(10)
name = 'usdcny'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

## 中美利差
fig,content = na.show_interest_rate_data(10)
name = 'interest_rate'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)


## 北向资金
fig,content = na.show_north_money_data(10)
name = 'north_money'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

In [7]:
# 2 流动性偏好
lq = Liquidity()

## 2.1 美元
### 美元指数
fig,content = lq.show_dollar_data(10)
name = 'DINIW'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

### 美日汇率
fig,content = lq.show_usdjpy_data(10)
name = 'usdjpy'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

### 美国十年国债收益率
fig,content = lq.show_us10yr_data(10)
name = 'us10yr'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

### 纳斯达克100指数
fig,content = lq.show_ndx_data(10)
name = 'NDX'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

### 美元计价大宗商品
fig,content = lq.show_uscl_data(10)
name = 'uscl'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

## 2.2 人民币
### 十年国债收益率
fig,content = lq.show_cn10yr_data(10)
name = 'cn10yr'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

#### 融资融券余额
l = Leverage()
fig,content = l.show_data()
name = 'leverage'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

#### 股票型公募基金仓位
fig,content = l.show_position_data()
name = 'fund_position'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

In [8]:
# 3 经济景气度
e = Economic(data_path)

## 3.1 美国
### US PMI
fig,content = e.show_us_pmi_data(12)
name = 'us_pmi'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

### US CPI
fig,content = e.show_us_cpi_data(12)
name = 'us_cpi'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

### 美国国债利差：10年-2年
fig,content = e.show_us_bond_diff_data(12)
name = 'us_bond_10Y-2Y'
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)


## 3.2 中国
## PMI
name = 'pmi'
fig,content = e.show_pmi_data(12)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)
### 新增信贷
name = 'xzxd'
fig,content = e.show_loan_data(12)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)
### PPI-CPI剪刀差
name = 'ppi-cpi'
fig,content = e.show_ppi_cpi_data(12)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)


In [9]:
# 4 市场倾向性
## 4.1 信用风险
d = Direction()
#### 大市值-小市值
name = 'credit_risk'
fig,content = d.show_risk_data(5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)
### 十年国债期货
name = 'treasury_bonds'
fig,content = e.show_bonds_data(5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

## 4.2 剩余流动性
### 小市值
name = 'liquidity'
fig,content = d.show_liquidity_data(5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)
### 沪深两市成交额
name = 'turnover'
fig,content = d.show_volume_data(5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

## 4.3 均衡与偏离
### 大市值
name = 'balance'
fig,content = d.show_balance_data(5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)
### 市场主线
name = 'mainline'
fig,content = d.show_main_data(5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)



In [10]:
# 5 市场情绪
s = Sentiment(data_path)
# 5.1 A股
### 上证50波幅指数
name = 'vix_50'
fig,content = s.show_ch_vix_data('510050',5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)
### 沪深300波幅指数
name = 'vix_300'
fig,content = s.show_ch_vix_data('510300',5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)
### 中证1000波幅指数
name = 'vix_852'
fig,content = s.show_ch_vix_data('000852',5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)

## 5.2 港股
### 恒指波幅指数
name = 'vix_hsi'
fig,content = s.show_vhsi_data(5)
img_data_dict = update_img_data_dict(img_data_dict,fig,name)
msg_content = repalce_message(msg_content,content,name,isEmail=True)
html_content = repalce_message(html_content,content,name,isEmail=False)



In [11]:
# 当前时间
datatime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
msg_content = msg_content.replace('当前时间',datatime)
html_content = html_content.replace('当前时间',datatime)

In [12]:

# 发送邮件
date = datetime.now().strftime("%Y-%m-%d")
subject = '每日回顾【{}】'.format(date)
# msh.sendMail(subject,message)
#     msh.sendMail(subject,msg_content,img_data_dict)

# 保存最新结果
with open(result_path + "html_content.html",'w',encoding='GBK') as f:
    f.write(html_content)

    # 保存最新结果
with open(result_path + "index.html",'w',encoding='UTF-8') as f:
    f.write(html_content)
# return html_content
# html_content = main()

In [37]:
!git push -u origin main

error: src refspec main does not match any.
error: failed to push some refs to 'git@github.com:NotCounted/macro.git'


In [ ]:
# github
# !git remote rm origin
# !git remote add origin git@github.com:NotCounted/macro.git
# !git pull origin main
!git add .
!git commit -m "update"
!git push -u origin main

From github.com:NotCounted/macro
 * branch            main       -> FETCH_HEAD
 * [new branch]      main       -> origin/main
lousr/share/jed/lib/textmode.slc                                         modeinfo                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [31]:
# # gitee
# !git remote rm origin
# !git remote add origin git@gitee.com:notcounted/macro.git
# !git pull origin master
# !git add *.png html_content.html index.html
# !git commit -m "update"
# !git push -u origin master

On branch main
Untracked files:
	.NDX.csv
	.NDX.png
	.ipynb_checkpoints/
	CN10YR.csv
	DINIW.csv
	OptData_000016.XSHG.csv
	OptData_000300.XSHG.csv
	OptData_000852.XSHG.csv
	OptData_510050.XSHG.csv
	OptData_510300.XSHG.csv
	SH000985.csv
	US10YR.csv
	USCL.csv
	USDCNY.csv
	USDJPY.csv
	Untitled.ipynb
	VIXData_000016.XSHG.csv
	VIXData_000300.XSHG.csv
	VIXData_000852.XSHG.csv
	VIXData_510050.XSHG.csv
	VIXData_510300.XSHG.csv
	cn_dr001.csv
	cn_dr007.csv
	content.html
	cpi.csv
	id-rsa
	id-rsa.pub
	loan.csv
	north_money.csv
	pmi.csv
	ppi.csv
	shibor_oneweek.csv
	us_bond_10Y-2Y.csv
	us_cpi.csv
	us_fed_rate.csv
	us_manufacturing_pmi.csv
	us_non_manufacturing_pmi.csv
	"\344\277\235\345\255\230\345\210\260git.ipynb"
	"\346\257\217\346\227\245\345\233\236\351\241\276.ipynb"

nothing added to commit but untracked files present
error: src refspec master does not match any.
error: failed to push some refs to 'git@gitee.com:notcounted/macro.git'


In [1]:
# github
!git init

Initialized empty Git repository in /home/jquser/0.定时推送/datasource/DailyReview/data/result/.git/


In [ ]:
!git pull origin main
!git add *.png html_content.html index.html
!git commit -m "update1"

In [37]:
!git status

On branch main
Your branch is up-to-date with 'origin/main'.
Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.NDX.csv
	.NDX.png
	.ipynb_checkpoints/
	CN10YR.csv
	DINIW.csv
	OptData_000016.XSHG.csv
	OptData_000300.XSHG.csv
	OptData_000852.XSHG.csv
	OptData_510050.XSHG.csv
	OptData_510300.XSHG.csv
	SH000985.csv
	US10YR.csv
	USCL.csv
	USDCNY.csv
	USDJPY.csv
	Untitled.ipynb
	VIXData_000016.XSHG.csv
	VIXData_000300.XSHG.csv
	VIXData_000852.XSHG.csv
	VIXData_510050.XSHG.csv
	VIXData_510300.XSHG.csv
	cn_dr001.csv
	cn_dr007.csv
	content.html
	cpi.csv
	id-rsa
	id-rsa.pub
	loan.csv
	north_money.csv
	pmi.csv
	ppi.csv
	shibor_oneweek.csv
	us_bond_10Y-2Y.csv
	us_cpi.csv
	us_fed_rate.csv
	us_manufacturing_pmi.csv
	us_non_manufacturing_pmi.csv
	"\344\277\235\345\255\230\345\210\260git.ipynb"
	"\346\257\217\346\227\245\345\233\236\351\241\276.ipynb"

nothing added to commit but untracked files present (use "git add" to track)


In [16]:

print(datatime)

2023-02-09 08:40:29


In [17]:
url = 'https://www.joinquant.com/user/74461734912/files/0.%E5%AE%9A%E6%97%B6%E6%8E%A8%E9%80%81/datasource/DailyReview/data/index.html'
print(url)

https://www.joinquant.com/user/74461734912/files/0.%E5%AE%9A%E6%97%B6%E6%8E%A8%E9%80%81/datasource/DailyReview/data/html_content.html
